### 랭체인을 통한 gpt 활용
1. 그냥 openai를 사용할 때와 객체 생성 함수가 다름
2. 랭체인은 LLM을 기반으로 AI를 더 쉽게 활용할 수 있는 프레임워크

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.tracers.stdout import ConsoleCallbackHandler

In [2]:
from dotenv import load_dotenv
load_dotenv()
openAPI_KEY = os.getenv('openAPI')

os.environ['OPENAI_API_KEY'] =openAPI_KEY

In [3]:
llm = ChatOpenAI(
    temperature=0.1, # 0으로 갈 수로 사실에 기반한 답변, 2는 거의 상상해서 답변(0~2)
    max_tokens=2048,
    model="gpt-4o"
)

In [4]:
# 질의 테스트
question = "세종대왕이 누구 인지 설명해주세요"

result = llm.invoke(question)
print(result.content)

세종대왕(1397년 5월 15일 ~ 1450년 3월 30일)은 조선의 제4대 왕으로, 조선 역사상 가장 존경받는 군주 중 한 명입니다. 본명은 이도이며, 조선 태종의 셋째 아들로 태어났습니다. 세종대왕은 그의 재위 기간 동안 여러 가지 중요한 업적을 남겼습니다.

가장 유명한 업적은 한글의 창제입니다. 세종대왕은 백성들이 쉽게 읽고 쓸 수 있는 문자를 만들고자 하여 1443년에 훈민정음을 창제하고, 1446년에 반포하였습니다. 이는 한국어를 표기하는 데 혁신적인 변화를 가져왔고, 오늘날까지도 사용되고 있습니다.

세종대왕은 또한 과학과 기술의 발전을 장려하였습니다. 그는 천문학, 농업, 의학 등 다양한 분야에서 연구와 발전을 지원하였으며, 장영실과 같은 뛰어난 과학자들을 등용하여 자격루, 앙부일구, 측우기 등의 발명을 이끌었습니다.

또한, 세종대왕은 국방과 외교 정책에서도 중요한 역할을 했습니다. 그는 북방의 여진족과의 관계를 안정시키고, 일본과의 교류를 통해 외교적 안정을 도모하였습니다.

세종대왕의 통치는 조선의 문화와 과학, 경제의 발전에 큰 기여를 하였으며, 그의 업적은 오늘날까지도 한국 사회에 깊은 영향을 미치고 있습니다.


### 템플릿을 이용하여 gpt에게 명령
1. 이런것 때문에 랭체인 활용
2. 사실 별건 없다. 더군다나 아래는 그냥 리스트 이름으로 프롬프트를 계속생성
3. 그리고 invoke 함수 호출

In [5]:
templete = "{who}가 누구인지 설명해주세요"

prompt = PromptTemplate(
    template=templete, input_variables=['who']
)

In [6]:
# llm과 프롬프트를 연결(체인)
llm_chain = prompt | llm

# 템플릿을 이용한 출력
alist = ['트럼프', '오바마']

for item in alist:
    result = llm_chain.invoke({"who":item})
    print(result.content)
    print()



도널드 존 트럼프(Donald John Trump)는 미국의 기업가이자 정치인으로, 2017년부터 2021년까지 제45대 미국 대통령을 역임했습니다. 그는 1946년 6월 14일 뉴욕에서 태어났으며, 부동산 개발업체인 트럼프 그룹(Trump Organization)의 회장으로서 사업 경력을 쌓았습니다. 트럼프는 또한 리얼리티 TV 프로그램 "어프렌티스(The Apprentice)"의 진행자로도 잘 알려져 있습니다.

정치적으로는 공화당 소속이며, 2016년 대통령 선거에서 힐러리 클린턴을 누르고 당선되었습니다. 그의 대통령 재임 기간 동안 여러 가지 논란과 정책 변화가 있었으며, 특히 이민 정책, 세제 개혁, 외교 정책 등에서 많은 주목을 받았습니다. 2020년 대선에서는 조 바이든에게 패배하였고, 이후 선거 결과에 대한 여러 논란이 있었습니다. 트럼프는 정치 무대에서 여전히 영향력 있는 인물로 남아 있으며, 그의 지지자들 사이에서 강력한 지지를 받고 있습니다.

버락 오바마(Barack Obama)는 미국의 정치인으로, 2009년부터 2017년까지 제44대 미국 대통령을 역임했습니다. 그는 미국 역사상 최초의 아프리카계 미국인 대통령입니다. 오바마는 1961년 8월 4일 하와이 호놀룰루에서 태어났으며, 하버드 로스쿨을 졸업한 후 변호사로 활동했습니다.

정치 경력으로는 일리노이주 상원의원(1997-2004)과 미국 연방 상원의원(2005-2008)을 지냈습니다. 2008년 대통령 선거에서 민주당 후보로 출마하여 당선되었고, 2012년 재선에 성공했습니다. 그의 주요 정책으로는 오바마케어로 알려진 건강보험 개혁, 경제 회복 노력, 기후 변화 대응, 그리고 외교 정책에서의 다자주의 강화 등이 있습니다.



### 과거의 메세지까지 담을 수 있는 챗봇 생성
1. 여러 사람이 접근하는데 한 사람이 전체 대화를 세션이라고 한다.

In [ ]:
# 템플릿 생성. 이걸 왜 한거지? 아래에서는 거의 쓰지 않는 듯 한데.
# 저자가 갈 수록 나태지는 구만
templete = """아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 위키독스봇입니다.
대화 문맥을 바탕으로 친절한 답변을 진행하세요.
Current Conversation:
{history}

Human: {input}
AI:"""

prompt = PromptTemplate(
    template=templete, input_variables=['history','input']
)

# llm을 생성하고 객체 연결
llm = ChatOpenAI(model="gpt-4o")
chain = prompt | llm

In [29]:
store={}
session_id = "test"

if session_id not in store:
    store[session_id] = ChatMessageHistory()
session_history = store[session_id]

In [30]:
with_message_history = RunnableWithMessageHistory( chain,
lambda session_id: session_history, input_messages_key="input",
history_messages_key="history" )

In [31]:
result = with_message_history.invoke(
    {"input": "당신은 어디에서 만들었습니까?"}, config={"configurable": {"session_id": "test"}},
)
print(result.content)

저는 여러 지식과 정보를 바탕으로 다양한 개발자들이 협력하여 만든 AI입니다. 어디 한 곳에서 만들어진 것이 아니라 여러 곳에서의 연구와 개발이 결합되어 완성되었답니다. 다른 궁금한 점이 있으면 언제든지 물어보세요!


In [32]:
result = with_message_history.invoke(
    {"input": "푸른 바다를 주제로 감성적이고 짧은 시를 하나 지어 주세요"}, config={"configurable": {"session_id": "test"}},
)
print(result.content)

푸른 바다에 물결이 춤추고  
햇살은 은빛으로 반짝이네.  
바람은 부드럽게 속삭이며  
내 마음 깊숙이 스며드네.


In [33]:
result = with_message_history.invoke(
    {"input":"석양을 주제로도 해줘"}, config={"configurable":{"session_id":"test"}},
)
print(result.content)

석양이 물든 하늘 아래,  
붉은 물결이 고요히 흐르네.  
나뭇잎은 금빛으로 물들고,  
하루의 끝자락에서 숨을 고르네.  


In [38]:
print(store)

{'test': InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 어디에서 만들었습니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 여러 지식과 정보를 바탕으로 다양한 개발자들이 협력하여 만든 AI입니다. 어디 한 곳에서 만들어진 것이 아니라 여러 곳에서의 연구와 개발이 결합되어 완성되었답니다. 다른 궁금한 점이 있으면 언제든지 물어보세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 66, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a0e9480a2f', 'id': 'chatcmpl-CtluqBiNR9j0MkI6WrQyzin6wISVc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b81bd-e4ba-7dc1-ae1e-5d7b53340b35-0', usage_metadata={'input_tokens': 66, 'output_tokens': 63, 'total_tokens': 129, 'input_token_details': {'audi